# TIMING II Pipeline Demo
Welcome to TIMING II Pipeline!  This notebook will walk you step by step through the whole TIMING Pipeline, which includes the nanowell detection, cell segmentationa, tracking and feature calculation. The visualization module works independently from another terminal.

# Imports

In [ ]:
import sys, time

## Env setup

In [ ]:
CORE_NUMBER = 8   #Multi-core speed up setting

TIMING_II_HOME = 'E:\\TIMING\\TIMING2\\'

############################################## Don't Modify below in this code cell ################################### 
# This is needed for nanowell detection.
sys.path.append(TIMING_II_HOME + 'timing2-preprocessing\\')

# This is needed for nanowell detection.
sys.path.append(TIMING_II_HOME + 'timing2-crop\\faster-rcnn\\')

# This is needed for nanowell cropping.
sys.path.append(TIMING_II_HOME + 'timing2-crop\\')

# This is needed for segmentation modules
sys.path.append(TIMING_II_HOME + 'timing2-seg\\')

# This is needed for Cell Tracking.
sys.path.append(TIMING_II_HOME + 'timing2-tracker\\')

# This is needed for Utility functions.
sys.path.append(TIMING_II_HOME + 'timing2-utils\\')

# This is needed for feature calculation modules
sys.path.append(TIMING_II_HOME + 'timing2-features\\')

## TIMING II Parameter Configuration
Set up the parameters below before running the process for a batch of input data.

In [ ]:
###### Part 1: Dataset Parameters
Dataset_Name = 'TEST'
Data_Raw_DIR = 'E:\\TIMING\\TIMING2_Datasets_Raw'
Data_DIR = 'E:\\TIMING\\TIMING2_Benchmark_Datasets'

Dataset_Input = 'IN'
Dataset_Output = 'OUT'

Dataset_Blocks = ['B'+str(i).zfill(3) for i in range(1,101)]
Dataset_Channels = ['CH0','CH1','CH2','CH3']                     # Channels in the experiment, don't need to be changed
Dataset_Frames = 73
Block_Size = 2048 #pixels
Nanowell_Size = 281 #pixels

### Part 2: Preprocessing Parameters
stack_tuple = ("bright_field","effectors","targets","death")     # channels that are stacked together

unmix_tuple = ("targets","effectors", "death") # channels that are used in unmixing  
unmix_tuple_clean = ("targets", "effectors")   # if channels in this tuple, preprocessor will look for umx_Bxxx
unmix_ratio_c2_c3 = 0.77                        # otherwise, will use Bxxx to generate new bg_Bxxx    
unmix_ratio_c3_c4 = 0.22

enhance_tuple = ("targets",)
min_pixel_value = 0
max_pixel_value = 2000

preprocess_tuple = ("effectors","targets")      # channels that have umx and back ground subtraction

channel_dict = {"bright_field":"c1_ORG",
                "effectors":"c4_ORG",
                "targets":"c3_ORG",
                "death":"c2_ORG",
                "beads_1":"--",
                "beads_2":"--"}

channel_naming_dict = {"bright_field":"0",
                       "effectors":"1",
                       "targets":"2",
                       "death":"3",
                       "beads_1":"4",
                       "beads_2":"5"}
microscope = 'zeiss'


### Part 3: Pipeline Parameters

# Channels Available
Channel_available = ['CH0', 'CH1', 'CH2', 'CH3']      # Available Channels in experiments

# Segmentation Radius for Channel 1 and 2
R_CH1 = 20   # 23
R_CH2 = 21   # 24

# Resegmentation Configuration
CELL_COUNT_THRESHOLD = 0.8
CELL_COUNT_MAX = 4

# TIMING II Pipeline Step by Step

## A. Preprocessing


Step 0: Make directory for preprocessing, only need to run one time

In [ ]:
from Preprocess_Wrapper import TIMING_Preprocess_Mkdir

TIMING_Preprocess_Mkdir(Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks)

In [ ]:
### Put all the Preprocessing in a block by block cycle
from Preprocess_Wrapper import TIMING_Preprocess_Stack
from Preprocess_Wrapper import TIMING_Preprocess_Unmix
from Preprocess_Wrapper import TIMING_Preprocess_BackgroundSubtract
from Preprocess_Wrapper import TIMING_Preprocess_Enhance


for Block in Dataset_Blocks:
        t1 = time.time()
        
        print("Start Pre-Processing " + Block + " ......")
        
        Blocks = [Block]
        
        # Stack the Images
        # TIMING_Preprocess_Stack(TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, stack_tuple, channel_dict, channel_naming_dict, microscope, CORE_NUMBER)

        # Unmixing c2_ORG and c3_ORG
        TIMING_Preprocess_Unmix(2, 0, unmix_ratio_c2_c3, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Unmixing c3_ORG and c4_ORG
        TIMING_Preprocess_Unmix(0, 1, unmix_ratio_c3_c4, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Background Subtraction
        TIMING_Preprocess_BackgroundSubtract(40, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, preprocess_tuple, stack_tuple, unmix_tuple_clean, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Channel Enhancement
        TIMING_Preprocess_Enhance(min_pixel_value, max_pixel_value,TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, enhance_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        print("Preprocessing " + Block +" TIME: " +str(time.time() - t1))



## B. Nanowell Localization, Classification and Cropping
 We use faster r-cnn to do nanowell localization and identify the empty ones which we will discard automatically. Small image patch will be cropped right after the localization.

In [ ]:
from Crop_Wrapper import TIMING_Crop

t1 = time.time()

TIMING_Crop(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, Nanowell_Size, Block_Size, Channel_available)

print("STEP-CROPPING TIME: " +str(time.time() - t1))

## C. Cell Segmentation 
We use i-vote to detect the seeds for cell candidates. And we use k-means to extract the foreground of cells from the fluorescent channels(CH1 and CH2). Individual cell bodies are clustered based on their distance from each detected seeds.

In [ ]:
 from Segment_Wrapper import TIMING_Segment

t1 = time.time()

Thresh_CH1= 0.4    # Needs to specify this threshold value for two channels, the lower the value, the more cell bodies

Thresh_CH2= 0.2

TIMING_Segment(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, Nanowell_Size, Block_Size, Channel_available, Thresh_CH1, Thresh_CH2, R_CH1, R_CH2)

print("STEP-CELL SEGMENTATION TIME: " +str(time.time() - t1))

## D. Confinement-Constrained Cell Resegmentation
Assume the numbers of cells detected are constant throughout the whole time line. 

In [ ]:
from Resegment_Wrapper import TIMING_Resegment

t1 = time.time()

TIMING_Resegment(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, CELL_COUNT_THRESHOLD, CELL_COUNT_MAX)

print("STEP-CELL RE-SEGMENTATION TIME: " +str(time.time() - t1)) 
        

## E. Cell Tracking
Based on the results from cell segmentation. We define the cost functions for cell track assignment with cell center distance measure, cell size difference measure and set distance measure. We try to find the optimal tracking result by minimizing the cost function.

In [ ]:
from Tracking_Wrapper2 import TIMING_Tracker2

t1 = time.time()

TIMING_Tracker2(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER)

print("STEP-CELL TRACKING TIME: " +str(time.time() - t1))

## F. Cell Feature Calculation
We quantify the cells in each identified tracks. For Effector cells in Channel 1, we calculate the centroid position, aspect ratio, speed and death marker level at each time point. For Target cells in Channel 2, we calculate an additional contact ratio.

In [ ]:
from Feature_Wrapper import TIMING_Features

t1 = time.time()

TIMING_Features(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER)

print("STEP-FEATURE CALCULATION TIME: " +str(time.time() - t1))